[View in Colaboratory](https://colab.research.google.com/github/mr-ubik/gans-from-theory-to-production/blob/master/EuroScipy2018.ipynb)

In [0]:
%ls

In [0]:
! git clone https://github.com/zurutech/gans-from-theory-to-production

In [0]:
%cd gans-from-theory-to-production/

In [0]:
!ls

In [0]:
!python prepare_celeba_dataset.py

In [0]:
!rm ngrok*

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
import numpy as np
import tensorflow as tf

import tensorboardcolab

tfgan = tf.contrib.gan

In [0]:
def deconv2d(inputs, filters, strides=(1, 1), activation=tf.nn.relu):
    """\"Deconvolution\" layer.
    
    It uses upsampling with nearest neighbor interpolation to reduce the
    presence of checkboard artifacts.
    """
    
    input_h, input_w = inputs.shape[1].value, inputs.shape[2].value
    layer_1 = tf.image.resize_nearest_neighbor(
        inputs, (2 * input_h, 2 * input_w), name="NNUpSample2D"
    )
    # Padding before convolution is used to reduce boundary artifacts
    layer_1 = tf.pad(layer_1, [[0, 0], [2, 2], [2, 2], [0, 0]], mode="CONSTANT")
    layer_2 = tf.layers.conv2d(
        inputs=layer_1,
        filters=filters,
        kernel_size=5,
        padding="valid",
        use_bias=False,
        activation=activation,
        strides=strides,
    )
    return layer_2

In [0]:
def generator_fn(inputs, mode):
    """Generator producing images from noise.

        Args:
            noise: A single Tensor representing noise.
            mode: tf.estimator.ModeKeys

        Returns:
            A 64x64 (None, 4096) flattened tensor whose values are
            inside the (-1, 1) range."""
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    linear = tf.layers.dense(inputs=inputs, units=1024 * 4 * 4, activation=tf.nn.relu)
    net = tf.reshape(linear, (-1, 4, 4, 1024))
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 512)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 256)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 128)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 64)
    net = tf.layers.conv2d(
        inputs=net,
        filters=3,
        kernel_size=5,
        padding="same",
        data_format="channels_last",
        use_bias=False,
        strides=(1, 1),)
    output = tf.tanh(net)
    return output

In [0]:
def custom_conv2d(inputs, filters, strides=(2,2)):
    """Helper layer used to instatiate `tf.layers.conv2d` with proper arguments."""
    layer_1 = tf.layers.conv2d(
        inputs=inputs,
        filters=filters,
        kernel_size=5,
        padding="same",
        data_format="channels_last",
        use_bias=False,
        strides=strides,
    )
    layer_1 = tf.nn.leaky_relu(layer_1, alpha=0.2)

    return layer_1

In [0]:
def discriminator_fn(inputs, conditioning, mode):
    """Build the Discriminator network.
    Args:
        features: a batch of images to classify, expected input shape (None, 64, 64 , 3)
        conditioning: a batch of labels, it is used for conditioning in the some model (es Conditional GAN).
            GANEstimator wants this parameters around, just define an arguments so that discriminator_fn is not broken.
        mode: tf.estimator.ModeKey
    
    Returns:
            The output (logits) of the discriminator.
    """
    
    # In every mode, define the model
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    net = custom_conv2d(inputs, filters=64)
    net = custom_conv2d(inputs, filters=128)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = custom_conv2d(net, filters=256)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = custom_conv2d(net, filters=512)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = custom_conv2d(net, filters=1024)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = tf.reshape(net, (-1, net.shape[1] * net.shape[2] * net.shape[3]))
    output = tf.layers.dense(net, units=1)

    return output

In [0]:
def _get_train_images_input_fn(file_pattern, image_size=(64, 64, 3), shuffle=False,
                 batch_size=32, num_epochs=None, buffer_size=4096):
    """get_input_fn exploits the `file_pattern` to create an input_fn that reads all the content
    of the specified pattern, creating an object dataset.
    
    Args:
        file_pattern: python string, the pattern of the file to read to generate the dataset
        image_size: the new size of the read images
        shuffle: True if the order of the elements in the generated dataset shold be randomized
        batch_size: the size of the batches
        num_epochs: the number of epochs to repeat the dataset before throwing an exeption; None is unlimited
        buffer_size: how many images read before starting to generate output
    Returns:
        input_fn: the generated input_fn that returns a correctly instantiated iterator
    """
    
    def _img_string_to_tensor(image_string):
        """Decode an image as read from a `tf.decode_raw`, scales it between 0-1 and resize the
        image as specified in the parent method.
        Args:
            image_string: the raw image tensor
        Returns:
            image_resize: image in [0,1] correctly resized
        """
        
        nonlocal image_size
        
        image_decoded = tf.image.decode_jpeg(image_string, channels=image_size[-1])
        # The conversion to float automatically scales the values in [0., 1.]
        image_decoded_as_float = tf.image.convert_image_dtype(image_decoded, dtype=tf.float32)
        image_decoded = (image_decoded_as_float - 0.5) * 2
        image_resized = tf.image.resize_images(image_decoded, size=image_size[:2])
        

        return image_resized

    def _path_to_img(path):
        """Given the path of an image, returns the pair (image, label)
        where image is the corretly resized image, and label is the label associated with it.
        Args:
            path: the path of the image to read
        Returns:
            (image_resized, label): the image, label pair associated the path
        """
        
        # Get the parent folder of this file to get its class
        # Associate the label 0 to dogs and 1 to cats
        label = tf.cond(
                    tf.equal(tf.string_split([path], delimiter='/').values[-2], "dogs"),
                    lambda: 0, lambda: 1)

        image_string = tf.read_file(path) # read image and process it
        image_resized = _img_string_to_tensor(image_string)

        return image_resized, label
    
    def _input_fn():
        """The input function that builds the `tf.data.Dataset` object and instantiate
        the iterator correctly ready to be use.
        Returns:
            the iterator associated to the built Dataset object.
        """
        
        # Use the static method `list_files` that builds a dataset of all
        # files matching this pattern.
        dataset_path = tf.data.Dataset.list_files(file_pattern)

        if shuffle:
            dataset_path = dataset_path.apply(tf.contrib.data.shuffle_and_repeat(buffer_size, num_epochs))
        else:
            dataset_path = dataset_path.repeat(num_epochs)

        # The map function maps the path to the pair (image, label)
        dataset = dataset_path.map(_path_to_img)
        dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))
        dataset = dataset.prefetch(buffer_size)
        
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()

    return _input_fn()[0]

In [0]:
def _get_train_input_fn(file_pattern, batch_size, num_epochs, noise_dims=100, **kwargs):
    def train_input_fn():
        real_data = _get_train_images_input_fn(
                 file_pattern,
                 batch_size=batch_size, 
                 num_epochs=num_epochs)
        noise = tf.random_normal([batch_size, noise_dims], name="train_noise")
        real_data.set_shape((batch_size,) + tuple(real_data.shape[1:]))
        print(noise, real_data)
        return noise, real_data
    return train_input_fn

In [0]:
def load_hyperparameters():
    hp = {
        "model_dir": "logs",
        "file_pattern": "assets/celeba/*.jpg",
        "batch_size": 128,
        "num_epochs":50,
        "noise_dims": 100
    }
    return hp

In [0]:
def dcgan():
    
    # Set the seed at a Graph Level so that we geet consistency between runs.
    tf.set_random_seed(42)
    
    # Hyperparameters
    hyperparameters = load_hyperparameters()
    
    # Run Configuration (it has other arguments)
    run_config = tf.estimator.RunConfig(
        model_dir=hyperparameters.get("model_dir"), save_summary_steps=50, save_checkpoints_steps=500)
    
    # Instatiate the GANEstimator object
    gan_estimator = tfgan.estimator.GANEstimator(
        config=run_config,
        generator_fn=generator_fn,
        discriminator_fn=discriminator_fn,
        generator_loss_fn=tfgan.losses.modified_generator_loss,
        discriminator_loss_fn=tfgan.losses.modified_discriminator_loss,
        generator_optimizer=tf.train.AdamOptimizer(0.0002, 0.5),
        discriminator_optimizer=tf.train.AdamOptimizer(0.0002, 0.5),
        add_summaries=tfgan.estimator.SummaryType.IMAGES
    )
    
    # Instatiate the train_input_fn
    # The model will train until it exhausts the Dataset which is repeated EPOCH times
    train_input_fn = _get_train_input_fn(**hyperparameters)
    trained_model = gan_estimator.train(train_input_fn, max_steps=None)
    return trained_model

In [0]:
def _predict_input_fn(batch_size, noise_dims=100, **kwargs):
    
    def predict_input_fn():
        noise_gen = np.array([np.float32(np.random.normal(size=[1, noise_dims])) for i in range(batch_size)])
        dataset = tf.data.Dataset.from_tensor_slices(noise_gen)
        dataset = dataset.batch(batch_size)
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()
        
    return predict_input_fn

In [0]:
trained_model = dcgan()

In [0]:
predictions_batch = 16
predict_input_fn = _predict_input_fn(batch_size=predictions_batch)
predictions = trained_model.predict(predict_input_fn)
new_celebs = [next(predictions) for _ in range(predictions_batch)]